In [1]:
import numpy as np
import pandas as pd
import random
import operator
from math import e
from math import pi
from sklearn.model_selection import KFold



In [2]:
iris=pd.read_csv('Iris.csv')
data=iris.iloc[:,[1,2,3,4]].values.tolist()
y=iris.iloc[:,5].values.tolist()






In [3]:
def group_class(data_list,x):
    
    group={}
    group['Iris-virginica']=[]
    group['Iris-setosa']=[]
    group['Iris-versicolor']=[]
    for i in data_list:
        if(y[i]=='Iris-setosa'):
            group['Iris-setosa'].append(data[i])
        elif(y[i]=='Iris-versicolor'):
            group['Iris-versicolor'].append(data[i])
        if(y[i]=='Iris-virginica'):
            group['Iris-virginica'].append(data[i])
    return group
    

In [4]:
def normal_pdf(x, mean, stdev):
        variance = stdev ** 2
        exp_squared_diff = (x - mean) ** 2
        exp_power = -exp_squared_diff / (2 * variance)
        exponent = e ** exp_power
        denominator = ((2 * pi) ** .5) * stdev
        normal_prob = exponent / denominator
        return normal_prob

In [5]:

def mean_data(data):
    return np.mean(data)

In [6]:
def std_data(data):
    return np.std(data)

In [7]:
def class_prob(group,target,data):
    return len(group[target])/(data)
    
    

In [8]:
def fun(features):
    for feature in zip(*features):
            yield {
                'stdev': std_data(feature),
                'mean': mean_data(feature)
            }

In [9]:
def joint_prob(row,summaries):
        joint_probs = {}
        for target, features in summaries.items():
            total_features = len(features['summary'])
            likelihood = 1
            for index in range(total_features):
                feature = row[index]
                mean = features['summary'][index]['mean']
                stdev = features['summary'][index]['stdev']
                normal_prob = normal_pdf(feature, mean, stdev)
                likelihood *= normal_prob
            prob = features['prob']
            joint_probs[target] =prob * likelihood
        return joint_probs

        
            
            

In [10]:
def predict(test,summaries):
    classes=[]
    for i in test:
        row=data[i]

        joint_probs=joint_prob(row,summaries)
        marginal_prob=sum(joint_probs.values())
        post_prob={}
#         print(joint_probs)
#         print(marginal_prob)
        for target,prob in joint_probs.items():
            post_prob[target]=prob/marginal_prob
#         print(post_prob)
        x=max(post_prob,key=post_prob.get)
       
        classes.append(x)
    return classes
            
    

In [11]:
def confusion_mat(classes,test_index,x):
   
    rows,cols=(3,3)
    n=0
    cm=[[0 for i in range(cols)] for j in range(rows)]
    for i in test_index:
        if(y[i]=='Iris-virginica'):
            if(classes[n]=='Iris-virginica'):
                cm[0][0]+=1
            elif(classes[n]=='Iris-setosa'):
                cm[0][1]+=1
            else:
                cm[0][2]+=1
            
        elif(y[i]=='Iris-setosa'):
            if(classes[n]=='Iris-virginica'):
                cm[1][0]+=1
            elif(classes[n]=='Iris-setosa'):
                cm[1][1]+=1
            else:
                cm[1][2]+=1
        else:
            if(classes[n]=='Iris-virginica'):
                cm[2][0]+=1
            elif(classes[n]=='Iris-setosa'):
                cm[2][1]+=1
            else:
                cm[2][2]+=1
        n+=1
    print("  Confusion Matrix")

    print(cm)
    sp=[]
    tp=[0 for i in range(3)]
    fp=[0 for i in range(3)]
    fn=[0 for i in range(3)]
    tn=[0 for i in range(3)]
    spp=[]
    tp[0]=cm[0][0]
    tp[1]=cm[1][1]
    tp[2]=cm[2][2]
    tn[0]=cm[1][1]+cm[2][2]
    tn[1]=cm[0][0]+cm[2][2]
    tn[2]=cm[0][0]+cm[1][1]
    fn[0]=cm[0][1]+cm[0][2]
    fn[1]=cm[1][0]+cm[1][2]
    fn[2]=cm[2][0]+cm[2][1]
    fp[0]=cm[1][0]+cm[2][0]
    fp[1]=cm[0][1]+cm[2][1]
    fp[2]=cm[1][2]+cm[0][2]
    for i in range(3):
        if tp[i]==0 and fn[i]==0:
            if fp[i]==0:
                sp.append(1.0)
            else:
                sp.append(0.0)
        else:
            x=tp[i]/(tp[i]+fn[i])
            sp.append(x)
    print("  Sensitivity is {}".format(np.mean(sp)))
  

    for i in range(3):
        if tn[i]==0 and fp[i]==0:
            if fn[i]==0:
                spp.append(1.0)
            else:
                spp.append(0.0)
        else:
            x=tn[i]/(tn[i]+fp[i])
            spp.append(x)
    print("  Specificity is {}".format(np.mean(spp)))
            

            


  
 
   

    
            

In [12]:
def accuracy(classes,test_list,x):
    corr=0
    n=0
    test_list=test_list.tolist()

    for i in test_list:
       
        if(y[i]==classes[n]):
            corr+=1
        n+=1
    return corr/len(test_list)
        
   

In [13]:
kf=KFold(n_splits=10, shuffle=False)
acc_mean=[]
x=0
for train_index, test_index in kf.split(iris):

    group=group_class(train_index,x)

    summaries = {}
    for target,features in group.items():
        summaries[target]={
            'prob':class_prob(group, target, len(train_index)),
            'summary': [i for i in fun(features)],
        }
    test=group_class(test_index,x)
    classes=predict(test_index,summaries)
    print("{} fold States------------->".format(x+1))
    
        
   
    
    acc=accuracy(classes,test_index,x)
    acc_mean.append(acc)
    
    confusion_mat(classes,test_index,x)
    print("  Accuracy is {}".format(acc*100))
    
    x+=1
    
print("     Average Accuracy of the model is {}".format(np.mean(acc_mean)*100))
    
    

       
    
   

    
        

1 fold States------------->
  Confusion Matrix
[[0, 0, 0], [0, 15, 0], [0, 0, 0]]
  Sensitivity is 1.0
  Specificity is 1.0
  Accuracy is 100.0
2 fold States------------->
  Confusion Matrix
[[0, 0, 0], [0, 15, 0], [0, 0, 0]]
  Sensitivity is 1.0
  Specificity is 1.0
  Accuracy is 100.0
3 fold States------------->
  Confusion Matrix
[[0, 0, 0], [0, 15, 0], [0, 0, 0]]
  Sensitivity is 1.0
  Specificity is 1.0
  Accuracy is 100.0
4 fold States------------->
  Confusion Matrix
[[0, 0, 0], [0, 5, 0], [1, 0, 9]]
  Sensitivity is 0.6333333333333333
  Specificity is 0.9777777777777779
  Accuracy is 93.33333333333333
5 fold States------------->
  Confusion Matrix
[[0, 0, 0], [0, 0, 0], [1, 0, 14]]
  Sensitivity is 0.6444444444444445
  Specificity is 0.6444444444444445
  Accuracy is 93.33333333333333
6 fold States------------->
  Confusion Matrix
[[0, 0, 0], [0, 0, 0], [2, 0, 13]]
  Sensitivity is 0.6222222222222222
  Specificity is 0.6222222222222222
  Accuracy is 86.66666666666667
7 fold Stat